In [13]:
import pandas as pd
import numpy as np
from openai import OpenAI
import pandas as pd
import numpy as np
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4")
from tqdm import tqdm
import pickle
from transformers import AutoModel, AutoTokenizer
from torch.nn.functional import cosine_similarity
import torch.nn.functional as F
import torch

model = 'llama3.1:70b'
num_ctx = 15000

In [14]:
pickle_file_path = 'AMS_Df.pkl'
# read the serialized DataFrame
with open(pickle_file_path, 'rb') as file:
    AMS_Df = pickle.load(file)
    


AMS_Df.head()

# get first row of the DataFrame
test = AMS_Df.iloc[0]["json_answer"]

In [23]:
test

{'Sinks': [['ActivityManagerService var101 = this.mAm;',
   'PackageManagerInternal var103 = var101.getPackageManagerInternalLocked();',
   'ResolveInfo var105 = var103.resolveService(param2,param4,var100,var19,param7);'],
  ['BatteryStatsService var323 = var322.mBatteryStatsService;',
   'BatteryStatsImpl var325 = var323.getActiveStatistics();',
   'ApplicationInfo var326 = var303.applicationInfo;'],
  ['IntentFirewall var421 = var420.mIntentFirewall;',
   'ComponentName var422 = var408.name;',
   'boolean var425 = var421.checkService(var422,param2,param7,param6,param4,var423);'],
  ['AppOpsManager.permissionToOpCode(var434);',
   'String var472 = AppOpsManager.opToName(var436);']]}

In [15]:
# Extract each array
arrays = test['Sinks']

for i, array in enumerate(arrays, 1):
    joined = '\n'.join(array)
    print(f"Array {i} (Joined):\n{joined}\n")

Array 1 (Joined):
ActivityManagerService var101 = this.mAm;
PackageManagerInternal var103 = var101.getPackageManagerInternalLocked();
ResolveInfo var105 = var103.resolveService(param2,param4,var100,var19,param7);

Array 2 (Joined):
BatteryStatsService var323 = var322.mBatteryStatsService;
BatteryStatsImpl var325 = var323.getActiveStatistics();
ApplicationInfo var326 = var303.applicationInfo;

Array 3 (Joined):
IntentFirewall var421 = var420.mIntentFirewall;
ComponentName var422 = var408.name;
boolean var425 = var421.checkService(var422,param2,param7,param6,param4,var423);

Array 4 (Joined):
AppOpsManager.permissionToOpCode(var434);
String var472 = AppOpsManager.opToName(var436);



In [16]:
AMS_Df.columns

Index(['EP', 'code', 'label', 'java_code', 'service_name', 'json_answer'], dtype='object')

In [ ]:
def process_json_answer(json_answer):
    global counter
    
    all = []
    try:
        arrays = json_answer['Sinks']
        for i, array in enumerate(arrays, 1):
            joined = '\n'.join(array)
            all.append(joined)
    except:    
        return []
    counter += 1
    return all

counter = 0
# Step 2: Apply to create a new column for joined json_answer
AMS_Df['sink_code'] = AMS_Df['json_answer'].apply(process_json_answer)
# print(counter)

AMS_Df.head()
# process_json_answer(test)

,EP,code,label,java_code,service_name,json_answer,sink_code,embeddings
0,bindService_ActivityManagerService_9,"[inv]: int var12 = bindIsolatedService(param2,...",1,// Method: bindService\n// Service: ActivityMa...,ActivityManagerService,{'Sinks': [['ActivityManagerService var101 = t...,[ActivityManagerService var101 = this.mAm;\nPa...,"[[tensor(0.0168), tensor(0.0319), tensor(-0.12..."
2,sendIdleJobTrigger_ActivityManagerService_1,[inv]: long var10 = Binder.clearCallingIdentit...,1,// Method: sendIdleJobTrigger\n// Service: Act...,ActivityManagerService,"{'Sinks': [['int var22 = broadcastIntent(0, va...","[int var22 = broadcastIntent(0, var19, 0, 0, 0...","[[tensor(0.0408), tensor(0.0519), tensor(-0.02..."
7,startService_ActivityManagerService_7,[inv]: boolean var12 = hasFileDescriptors();<|...,1,// Method: startService\n// Service: ActivityM...,ActivityManagerService,{'Sinks': [['boolean var425 = var421.checkServ...,"[boolean var425 = var421.checkService(var422,p...","[[tensor(0.0165), tensor(0.0207), tensor(-0.15..."
10,updateConfiguration_ActivityManagerService_2,[get]: ActivityTaskManagerService var3 = this....,1,// Method: updateConfiguration\n// Service: Ac...,ActivityManagerService,{'Sinks': [['SystemProperties.set(persist.sys....,"[SystemProperties.set(persist.sys.locale,var70...","[[tensor(-0.1117), tensor(-0.0383), tensor(0.0..."
22,getProcessMemoryInfo_ActivityManagerService_2,[inv]: long var6 = SystemClock.uptimeMillis();...,1,// Method: getProcessMemoryInfo\n// Service: A...,ActivityManagerService,"{'Sinks': [['Debug.getMemoryInfo(var60,var58);...","[Debug.getMemoryInfo(var60,var58);\nvar64.set(...","[[tensor(-0.0954), tensor(0.0144), tensor(-0.1..."


In [28]:
# display the first row first column of the DataFrame
AMS_Df.iloc[0]["sink_code"]

['ActivityManagerService var101 = this.mAm;\nPackageManagerInternal var103 = var101.getPackageManagerInternalLocked();\nResolveInfo var105 = var103.resolveService(param2,param4,var100,var19,param7);',
 'BatteryStatsService var323 = var322.mBatteryStatsService;\nBatteryStatsImpl var325 = var323.getActiveStatistics();\nApplicationInfo var326 = var303.applicationInfo;',
 'IntentFirewall var421 = var420.mIntentFirewall;\nComponentName var422 = var408.name;\nboolean var425 = var421.checkService(var422,param2,param7,param6,param4,var423);',
 'AppOpsManager.permissionToOpCode(var434);\nString var472 = AppOpsManager.opToName(var436);']

In [29]:
checkpoint="Salesforce/codet5p-110m-embedding"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
model = AutoModel.from_pretrained(checkpoint, trust_remote_code=True).to("cuda")
def get_code_embedding(code_snippet):
    """
    Generates embeddings for a given code snippet using a pre-trained model.

    Parameters:
    - code_snippet (str): The code for which embeddings are to be generated.
    - checkpoint (str): The model checkpoint to be used for embedding. Default is Salesforce/codet5p-110m-embedding.
    - device (str): Device to run the model on, either 'cuda' for GPU or 'cpu' for CPU. Default is 'cuda'.

    Returns:
    - torch.Tensor: Embedding tensor for the input code.
    """
    inputs = tokenizer.encode(code_snippet, return_tensors="pt").to("cuda")
    with torch.no_grad():
        embedding = model(inputs)[0]
    
    return embedding.cpu()


In [31]:
AMS_Df["embeddings"] = None
for index, row in tqdm(AMS_Df.iterrows(), total=len(AMS_Df), desc="Processing Rows"):
    code_snippets = row["sink_code"]
    embeddings = []
    for each in code_snippets:
        code_embedding = get_code_embedding(each)
        embeddings.append(code_embedding)
    AMS_Df.at[index, "embeddings"] = embeddings
    
# store using pickle 
# with open('AMS_Df_embeddings.pkl', 'wb') as file:
#     pickle.dump(AMS_Df, file)


Processing Rows: 100%|██████████| 192/192 [00:04<00:00, 40.43it/s]


In [21]:
AMS_Df.columns


Index(['EP', 'code', 'label', 'java_code', 'service_name', 'json_answer',
       'sink_code', 'embeddings'],
      dtype='object')

Compute Cosine Similarity

In [ ]:
F.cosine_similarity(embedding2, embedding3, dim=0)

In [ ]:

similarities = {}


for index1, row1 in tqdm(AMS_Df.iterrows(), total=len(AMS_Df), desc="Computing Similarities"):
    ep1_id = row1["EP"]  
    ep1_embeddings = row1["embeddings"]
    ep1_sink_code = row1["sink_code"]  
    
    closest_ep = None
    max_similarity = -1
    similar_sink_pair = None
    
    for index2, row2 in AMS_Df.iterrows():
        if index1 == index2:
            continue  
        
        ep2_id = row2["EP"]
        ep2_embeddings = row2["embeddings"]
        ep2_sink_code = row2["sink_code"]  
        
        # Compute pairwise similarities between all embeddings of EP1 and EP2
        for i, emb1 in enumerate(ep1_embeddings):
            for j, emb2 in enumerate(ep2_embeddings):
                sim = torch.nn.functional.cosine_similarity(torch.tensor(emb1), torch.tensor(emb2), dim=0).item()
                if sim > max_similarity:
                    max_similarity = sim
                    closest_ep = ep2_id
                    similar_sink_pair = (ep1_sink_code[i], ep2_sink_code[j])  
    
    similarities[ep1_id] = {
        "closest_ep": closest_ep,
        "max_similarity": max_similarity,
        "similar_sink_pair": similar_sink_pair
    }





Computing Similarities:   0%|          | 0/192 [00:00<?, ?it/s]/tmp/ipykernel_3312478/2481098310.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sim = torch.nn.functional.cosine_similarity(torch.tensor(emb1), torch.tensor(emb2), dim=0).item()
Computing Similarities: 100%|██████████| 192/192 [00:13<00:00, 14.70it/s]


In [38]:
for ep, details in similarities.items():
    if details['similar_sink_pair'] is None or details['closest_ep'] is None:
        print(f"EP {ep} has no similar EPs.")
    else:
        print(f"EP {ep} is most similar to EP {details['closest_ep']} with similarity {details['max_similarity']:.4f}")
        print(f"Similar sink codes:\nEP {ep}: \n{details['similar_sink_pair'][0]}\nEP \n{details['closest_ep']}: \n{details['similar_sink_pair'][1]}")
    print("\n\n")

EP bindService_ActivityManagerService_9 is most similar to EP startService_ActivityManagerService_7 with similarity 0.9424
Similar sink codes:
EP bindService_ActivityManagerService_9: 
ActivityManagerService var101 = this.mAm;
PackageManagerInternal var103 = var101.getPackageManagerInternalLocked();
ResolveInfo var105 = var103.resolveService(param2,param4,var100,var19,param7);
EP 
startService_ActivityManagerService_7: 
ResolveInfo var105 = var103.resolveService(param2,param4,var100,var19,param7);
PackageManagerInternal var103 = var101.getPackageManagerInternalLocked();
ActivityManagerService var101 = this.mAm;



EP sendIdleJobTrigger_ActivityManagerService_1 is most similar to EP broadcastIntent_ActivityManagerService_14 with similarity 1.0000
Similar sink codes:
EP sendIdleJobTrigger_ActivityManagerService_1: 
int var768 = getUidFromIntent(var21);
ProcessList var769 = this.mProcessList;
UidRecord var771 = var769.getUidRecordLocked(var768);
var771.updateHasInternetPermission();
EP 
b

In [39]:
similarities = {}

for index1, row1 in tqdm(AMS_Df.iterrows(), total=len(AMS_Df), desc="Computing Similarities"):
    ep1_id = row1["EP"]  
    ep1_embeddings = row1["embeddings"]
    ep1_sink_code = row1["sink_code"]  
    
    similar_sink_pairs = []  # List to store all similar sink code pairs with similarity > 0.8
    
    for index2, row2 in AMS_Df.iterrows():
        if index1 == index2:
            continue  
        
        ep2_id = row2["EP"]
        ep2_embeddings = row2["embeddings"]
        ep2_sink_code = row2["sink_code"]  
        
        # Compute pairwise similarities between all embeddings of EP1 and EP2
        for i, emb1 in enumerate(ep1_embeddings):
            for j, emb2 in enumerate(ep2_embeddings):
                sim = torch.nn.functional.cosine_similarity(torch.tensor(emb1), torch.tensor(emb2), dim=0).item()
                if sim > 0.8:  # Only consider pairs with similarity > 0.8
                    similar_sink_pairs.append({
                        "similarity": sim,
                        "ep1_code": ep1_sink_code[i],
                        "ep2_id": ep2_id,
                        "ep2_code": ep2_sink_code[j]
                    })
    
    # Store all similar pairs for the current EP
    similarities[ep1_id] = similar_sink_pairs

Computing Similarities:   0%|          | 0/192 [00:00<?, ?it/s]/tmp/ipykernel_3312478/1371509802.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sim = torch.nn.functional.cosine_similarity(torch.tensor(emb1), torch.tensor(emb2), dim=0).item()
Computing Similarities: 100%|██████████| 192/192 [00:14<00:00, 13.51it/s]


In [40]:
# Display results
for ep, similar_pairs in similarities.items():
    if not similar_pairs:
        print(f"EP {ep} has no similar EPs with similarity > 0.8.")
    else:
        print(f"EP {ep} has the following similar sink code pairs with similarity > 0.8:")
        for pair in similar_pairs:
            print(f"  - Similarity: {pair['similarity']:.4f}")
            print(f"    EP {ep}: \n{pair['ep1_code']}")
            print(f"    EP {pair['ep2_id']}: \n{pair['ep2_code']}")
            print()
    print("\n\n")

EP bindService_ActivityManagerService_9 has the following similar sink code pairs with similarity > 0.8:
  - Similarity: 0.8293
    EP bindService_ActivityManagerService_9: 
ActivityManagerService var101 = this.mAm;
PackageManagerInternal var103 = var101.getPackageManagerInternalLocked();
ResolveInfo var105 = var103.resolveService(param2,param4,var100,var19,param7);
    EP startService_ActivityManagerService_7: 
boolean var425 = var421.checkService(var422,param2,param7,param6,param4,var423);
ActivityManagerService var420 = this.mAm;
IntentFirewall var421 = var420.mIntentFirewall;

  - Similarity: 0.9424
    EP bindService_ActivityManagerService_9: 
ActivityManagerService var101 = this.mAm;
PackageManagerInternal var103 = var101.getPackageManagerInternalLocked();
ResolveInfo var105 = var103.resolveService(param2,param4,var100,var19,param7);
    EP startService_ActivityManagerService_7: 
ResolveInfo var105 = var103.resolveService(param2,param4,var100,var19,param7);
PackageManagerInternal

In [47]:
for ep, similar_pairs in similarities.items():
    if not similar_pairs:
        print(f"EP {ep} has no similar EPs with similarity > 0.8.")
    else:
        print(f"EP {ep} has the following similar sink code pairs with similarity > 0.8:")
        for pair in similar_pairs:
            print(f"  - Similarity: {pair['similarity']:.4f}")
            print(f"    EP {ep}")
            print(f"    EP {pair['ep2_id']}")
            print()
    print("\n\n")

EP bindService_ActivityManagerService_9 has the following similar sink code pairs with similarity > 0.8:
  - Similarity: 0.8293
    EP bindService_ActivityManagerService_9
    EP startService_ActivityManagerService_7

  - Similarity: 0.9424
    EP bindService_ActivityManagerService_9
    EP startService_ActivityManagerService_7

  - Similarity: 0.8829
    EP bindService_ActivityManagerService_9
    EP startService_ActivityManagerService_7

  - Similarity: 0.8558
    EP bindService_ActivityManagerService_9
    EP startService_ActivityManagerService_7

  - Similarity: 0.8993
    EP bindService_ActivityManagerService_9
    EP startService_ActivityManagerService_7

  - Similarity: 0.8860
    EP bindService_ActivityManagerService_9
    EP bindIsolatedService_ActivityManagerService_10

  - Similarity: 0.8776
    EP bindService_ActivityManagerService_9
    EP bindIsolatedService_ActivityManagerService_10

  - Similarity: 0.8860
    EP bindService_ActivityManagerService_9
    EP stopService_Ac

In [51]:

for ep, similar_pairs in similarities.items():
    if not similar_pairs:
        print(f"EP {ep} has no similar EPs with similarity > 0.8.")
    else:
        print(f"EP {ep} has the following similar sink code pairs with similarity > 0.8:")
        for pair in similar_pairs:
            ep2_id = pair['ep2_id']
            unique_ep2_ids = set()
            unique_ep2_ids.add(ep2_id)  # Add to set for uniqueness
            # print(f"  - Similarity: {pair['similarity']:.4f}")
            # print(f"    EP {ep}")
            # print(f"    EP {ep2_id}")
            # print()

        # Print unique EP2 IDs encountered so far
        print("Unique EP2 IDs so far:")
        for ep2_id in unique_ep2_ids:
            print(ep2_id)
    print("\n\n")


EP bindService_ActivityManagerService_9 has the following similar sink code pairs with similarity > 0.8:
Unique EP2 IDs so far:
startActivity_ActivityManagerService_11



EP sendIdleJobTrigger_ActivityManagerService_1 has the following similar sink code pairs with similarity > 0.8:
Unique EP2 IDs so far:
registerReceiver_ActivityManagerService_8



EP startService_ActivityManagerService_7 has the following similar sink code pairs with similarity > 0.8:
Unique EP2 IDs so far:
isUserAMonkey_ActivityManagerService_1



EP updateConfiguration_ActivityManagerService_2 has no similar EPs with similarity > 0.8.



EP getProcessMemoryInfo_ActivityManagerService_2 has the following similar sink code pairs with similarity > 0.8:
Unique EP2 IDs so far:
getProcessPss_ActivityManagerService_2



EP bindIsolatedService_ActivityManagerService_10 has the following similar sink code pairs with similarity > 0.8:
Unique EP2 IDs so far:
stopServiceToken_ActivityManagerService_4



EP stopService_ActivityM

In [46]:
print(AMS_Df[AMS_Df["EP"] == "updateConfiguration_ActivityManagerService_2"]["sink_code"].tolist())

[['SystemProperties.set(persist.sys.locale,var70);\nLocaleList.setDefault(var45,var63);', 'Settings$System.clearConfiguration(var15);\nActivityTaskManagerService$UpdateConfigurationResult var35 = this.mTmpUpdateConfigurationResult;\nboolean var37 = updateConfigurationLocked(var15,0,0,0,-10000,0,var35);', 'UsageStatsManagerInternal var114 = this.mUsageStatsInternal;\nvar114.reportConfigurationChange(var115,var118);', 'AttributeCache var123 = AttributeCache.instance();\nvar123.updateConfiguration(var124);', 'ActivityThread var128 = this.mSystemThread;\nvar128.applyConfigurationToResources(var129);']]
